#Packages & Global Variables

Install axelrod. Then run the imports.

In [ ]:
!pip install axelrod
!pip install pandas
!pip install openpyxl

In [ ]:
import axelrod as axl
from axelrod.action import Action, actions_to_str
from axelrod.player import Player
from axelrod.strategy_transformers import (
    FinalTransformer,
    TrackHistoryTransformer,
)
import random
import secrets
import pandas as pd
import openpyxl
import time

C, D = Action.C, Action.D

ITERATIONS = [1000, 10000]  # Use ITERATIONS[0] for now; ITERATIONS is the number of times we cycle between search method and playing against TFT
TURNS = 100                 # Number of turns for the match between search method strategy vs TFT
DEPTH = [3, 4, 5]           # Use DEPTH[0] for now
SHEETS = 0

# Axelrod citation:
# @misc{axelrodproject,
#   author       = {{ {The Axelrod project developers} }},
#   title        = {Axelrod: <RELEASE TITLE>},
#   month        = apr,
#   year         = 2016,
#   doi          = {<DOI INFORMATION>},
#   url          = {http://dx.doi.org/10.5281/zenodo.<DOI NUMBER>}
# }

## Template

[NEW] Template to follow in order to create and use strategies along with the player history. Copy & paste this to make your strategies with the search methods.

In [ ]:
class TestStrategy(Player):
    name = "Test Strategy"
    classifier = {
        "memory_depth": 3,
        "stochastic": False,
        "long_run_time": False,
        "inspects_source": False,
        "manipulates_source": False,
        "manipulates_state": False,
    }

    def __init__(self, memory):
        super().__init__()
        self.memory = memory
        self.strategy_size = 4 ** memory
        self.strategy_list = list(random.choice([Action.C, Action.D]) for i in range(self.strategy_size))           # initial strategy is random
        # self.strategy_list = list(Action.C if i % 2 == 0 else Action.D for i in range(self.strategy_size))        # TFT as a strategy array

    def moveProbabilities(self):
        c_count, d_count = 0.0, 0.0
        for i in self.strategy_list:
            if i == Action.C:
                c_count += 1
            else:
                d_count += 1

        return {'C': c_count / self.strategy_size, 'D': d_count / self.strategy_size}

    def strategyTable(self):                                                    # print the table like in the slides
        offset = self.memory * 2
        i_header = "Index"
        b_header = format("Binary", '^' + str(offset))
        h_header = format("History", '^' + str(offset))
        n_m_header = "Next Move"
        header = f"| {i_header} | {b_header} | {h_header} | {n_m_header} |"
        print(header)
        print("-" * len(header))

        for index,move in enumerate(self.strategy_list):
            binary = format(index, '0' + str(offset) + 'b')                     # convert the index to a string of binary with leading zeros
            history = "".join("C" if i == '0' else "D" for i in binary)         # convert the binary into its history counterpart
            print(f"| {index:>5} | {binary} | {format(history, '>' + str(len(h_header)))} | {move:^9} |")    # print row

    def nextMove(self, opponent):
        fullHist = self.history._plays[-3:] + opponent.history._plays[-3:]      # combine together the last 3 moves of self and opponent
        decimal = 0                                                             # index of the array
        place = 0                                                               # place of the binary value starting from the right
        for i in fullHist[::-1]:                                                # iterates the full history backwards (from right to left)
            if i == D:
                decimal += (2 ** place)                                         # D = 1, C = 0, so we only look at digits with D and add its decimal equivalent
            place += 1                                                          # moves to the next digit (right to left)

        return self.strategy_list[decimal]                                      # returns the next move

    def strategy(self, opponent: Player) -> Action:
        # if not self.history:                                                  # very first move
        #     return C

        # if len(self.history) < self.memory:                                   # assigns specific actions for the first few moves
        #     return D if opponent.history[-1] == C else D # the longer the consecutive action is, the more chances of getting stuck in the same move

        return self.nextMove(opponent)

    def changeStrategy(self):
                # call this method from the strategy() method
                # add your search method here; you can make multiple methods if you'd like
                # no need to return anything since strategy_list is part this class

                # if you want to keep track of what you've changed, you can either:
                # a.) return the index the search method modified and save it outside of this class
                # b.) create a new list at the top of this class like strategy_list and update it here
        pass    # closes the empty method; remove this when you finish writing the method

player = TestStrategy(DEPTH[0])                                                 # change how far back the strategy will look at (default: DEPTH[0] = 3)
battle = axl.Match((player, axl.TitForTat()), turns=10, seed=123)               # use axl.Match() for 1v1's against TFT

# player2 = TestStrategy(DEPTH[0])                                              # we may want to run another instance of our search method strategy to train
# battle2 = axl.Match((player2, axl.Cooperator()), turns=10, seed=123)          # against different strategies along with TFT and compare the results
# player3 = TestStrategy(DEPTH[0])                                              # the seed=123 helps us replicate the same starting strategy and see how different
# battle3 = axl.Match((player3, axl.Defector()), turns=10, seed=123)            # the strategies end up as


# tourney = axl.Tournament(player, axl.TitForTat(), axl.Cooperator, axl.Defector()) # once the strategy is formed, we compete it with other strategies to find which one gives us the best result in the end
# t_result = tourney.play()

result = battle.play()
# we may need to put the play() in a for loop depending on the search method
print(result)
# player.strategyTable()                                         # prints the table of index, binary, history, and next move of the strategy like in the slides
print(player.moveProbabilities())



[(C, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C)]
{'C': 0.625, 'D': 0.375}


# Hill Climbing

In [ ]:
strategy_size = 4 ** DEPTH[0]


class HillClimbing(Player):

    name = 'HillClimbing'
    classifier = {
        'memory_depth': 3,
        'stochastic': False,
        'inspects_source': False,
        'manipulates_source': False,
        'manipulates_state': False
    }

    def __init__(self, memory):
        super().__init__()
        self.memory = memory
        self.strategy_list = list(secrets.choice(
            [C, D]) for i in range(strategy_size))
        self.score = 0

    # print the table like in the slides
    def strategyTable(self):
        offset = self.memory * 2
        i_header = "Index"
        b_header = format("Binary", '^' + str(offset))
        h_header = format("History", '^' + str(offset))
        n_m_header = "Next Move"
        result_header = "Result"
        header = f"| {i_header} | {b_header} | {h_header} | {n_m_header} |"

        print(header)
        print("-" * len(header))
        for index, move in enumerate(self.strategy_list):
            # convert the index to a string of binary with leading zeros
            binary = format(index, '0' + str(offset) + 'b')
            # convert the binary into its history counterpart
            history = "".join("C" if i == '0' else "D" for i in binary)
            print(
                f"| {index:>5} | {binary} | {format(history, '>' + str(len(h_header)))} | {move:^9} |")

    def nextMove(self, opponent):
        # combine together the last 3 moves of self and opponent
        fullHist = self.history._plays[-3:] + opponent.history._plays[-3:]
        # index of the array
        decimal = 0
        # place of the binary value starting from the right
        place = 0
        # iterates the full history backwards (from right to left)
        for i in fullHist[::-1]:
            if i == D:
                # D = 1, C = 0, so we only look at digits with D and add its decimal equivalent
                decimal += (2 ** place)
            # moves to the next digit (right to left)
            place += 1

        return self.strategy_list[decimal]

    def strategy(self, opponent):

        if len(self.history) >= 3:
            return self.nextMove(opponent)
        else:
            return secrets.choice([C, D])


def generateNeighbors(memory, neighbors):
    return list(HillClimbing(memory) for i in range(neighbors))


def bestNeighbor(population):
    tournament = axl.Tournament(population, turns=1, repetitions=1)
    results = tournament.play()
    winner_index = results.ranking[0]
    population[winner_index].score = results.scores[0]
    return population[winner_index]


for i in range(1000):
    print(i)
    if i == 0:
        current_solution = bestNeighbor(generateNeighbors(DEPTH[0], 100))
    else:
        best_neighbor = bestNeighbor(generateNeighbors(DEPTH[0], 100))
        if current_solution.score < best_neighbor.score:
            current_solution = best_neighbor
        if random.uniform(0, 1) <= 0.00:  # Adjust decimal for probability
            current_solution = best_neighbor
print(current_solution.score)
current_solution.strategyTable()


# Local Beam Search

ignore the first one

In [ ]:
from typing import List, Tuple
from random import choices
import axelrod as axl

def local_beam_search(initial_state: Tuple[int, int, int], actions: List[int],
                      opponent_actions: List[int], beam_width: int) -> List[Tuple[int, int]]:
    """
    Perform a local beam search for the best next move given a beam width.
    """
    def possible_actions(state: Tuple[int, int, int], actions: List[int], opponent_actions: List[int]) -> List[Tuple[int, int, int]]:
        """
        Get the possible next states and corresponding actions.
        """
        states = []
        for action in actions:
            new_state = (state[1], state[2], action)
            states.append((new_state, action))
        return states

    def evaluate(state: Tuple[int, int, int], opponent_actions: List[int]) -> float:
        """
        Evaluate the current state by comparing it to the opponent's actions.
        """
        score = 0
        for action in opponent_actions:
            if state[2] == action:
                score += 1
        return score

    states = possible_actions(initial_state, actions, opponent_actions)
    scores = [evaluate(state, opponent_actions) for state, _ in states]
    sorted_states = [state for _, state in sorted(zip(scores, states), key=lambda x: x[0], reverse=True)]
    return sorted_states[:beam_width]

player1, player2 = axl.TitForTat(), axl.Random()
battle = axl.Match((player1, player2), turns=10)
result = battle.play()
print(result)

actions = [0, 1]  # 0 is C and 1 is D
opponent_actions = result[1]
initial_state = (0, 0, 0)
beam_width = 3

best_actions = local_beam_search(initial_state, actions, opponent_actions, beam_width)
print(best_actions)

In [ ]:
import random
import numpy as np

def tft_strategy(history):
    if len(history) == 0:
        return 'C'
    else:
        return history[-1]

def random_strategy(history):
    return random.choice(['C', 'D'])

def local_beam_search_strategy(history, n=2, k=1):
    strategies = [random_strategy(history) for _ in range(n)]
    scores = [calculate_score(history, strategy) for strategy in strategies]
    best_strategies = [strategies[i] for i in np.argsort(scores)[-k:]]
    return random.choice(best_strategies)

def calculate_score(history, strategy):
    opponent_strategy = tft_strategy(history)
    if strategy == 'C' and opponent_strategy == 'C':
        return 3
    elif strategy == 'D' and opponent_strategy == 'C':
        return 5
    elif strategy == 'C' and opponent_strategy == 'D':
        return 0
    elif strategy == 'D' and opponent_strategy == 'D':
        return 1

def simulate_game(strategy1, strategy2, rounds=100):
    history1 = []
    history2 = []
    scores1 = 0
    scores2 = 0
    for i in range(rounds):
        action1 = strategy1(history2)
        action2 = strategy2(history1)
        history1.append(action1)
        history2.append(action2)
        scores1 += calculate_score(history2, action1)
        scores2 += calculate_score(history1, action2)
    return (scores1, scores2)

if __name__ == '__main__':
    results = [simulate_game(local_beam_search_strategy, tft_strategy) for _ in range(100)]
    avg_result = np.mean(results, axis=0)
    print("Average score for Local Beam Search:", avg_result[0])
    print("Average score for TFT:", avg_result[1])


In [ ]:
from collections import defaultdict
import itertools
import numpy as np
import axelrod as axl

class TestStrategy(axl.Player):
    name = "Test Strategy"
    classifier = {
        "memory_depth": 3,
        "stochastic": False,
        "long_run_time": False,
        "inspects_source": False,
        "manipulates_source": False,
        "manipulates_state": False,
    }

    def __init__(self, memory, beam_size=10):
        super().__init__()
        self.memory = memory
        self.strategy_size = 4 ** memory
        self.strategy_list = [random.choice([axl.Action.C, axl.Action.D]) for i in range(self.strategy_size)]
        self.beam_size = beam_size


    def changeStrategy(self, new_strategy):
        self.strategy_list = new_strategy

    def strategyTable(self):
        offset = self.memory * 2
        i_header = "Index"
        b_header = format("Binary", '^' + str(offset))
        h_header = format("History", '^' + str(offset))
        n_m_header = "Next Move"
        header = f"| {i_header} | {b_header} | {h_header} | {n_m_header} |"
        print(header)
        print("-" * len(header))
        for index,move in enumerate(self.strategy_list):
            binary = format(index, '0' + str(offset) + 'b')
            history = "".join("C" if i == '0' else "D" for i in binary)
            print(f"| {index:>5} | {binary} | {format(history, '>' + str(len(h_header)))} | {move:^9} |")

    def moveProbabilities(self):
        unique, counts = np.unique(self.strategy, return_counts=True)
        return dict(zip(unique, counts))

    def nextMove(self, opponent):
        fullHist = self.history._plays[-3:] + opponent.history._plays[-3:]      # combine together the last 3 moves of self and opponent
        decimal = 0                                                             # index of the array
        place = 0                                                               # place of the binary value starting from the right
        for i in fullHist[::-1]:                                                # iterates the full history backwards (from right to left)
            if i == D:
                decimal += (2 ** place)                                         # D = 1, C = 0, so we only look at digits with D and add its decimal equivalent
            place += 1                                                          # moves to the next digit (right to left)

        return self.strategy_list[decimal]

    def generate_neighbors(self):
        for i in range(len(self.strategy)):
          new_strategy = self.strategy.copy()
          new_strategy[i] = 1 - new_strategy[i]
          yield new_strategy


    def strategy(self, opponent: Player) -> Action:
        return self.nextMove(opponent)

    def evaluate_strategy(self, rounds=100):
      total_score = 0
      for i in range(rounds):
        p1 = axl.Cooperator()
        p2 = axl.Defector()
        match = axl.Match((self, p1), turns=100)
        result = match.play()
        total_score += result[0][self.ID - 1]

        p1 = axl.Defector()
        p2 = axl.Cooperator()
        match = axl.Match((self, p1), turns=100)
        result = match.play()
        total_score += result[0][self.ID - 1]
      return total_score / (2 * rounds)



    def local_beam_search(self, rounds=100):
      scores = defaultdict(int)
      for neighbor in self.generate_neighbors():
        self.changeStrategy(neighbor)
        scores[tuple(neighbor)] = self.evaluate_strategy(rounds)
      best_neighbors = sorted(scores, key=lambda x: scores[x], reverse=True)[:self.beam_size]
      next_strategy = random.choice(best_neighbors)
      self.changeStrategy(next_strategy)
      return next_strategy


#player = TestStrategy(DEPTH[0])                                                 # change how far back the strategy will look at (default: DEPTH[0] = 3)
#battle = axl.Match((player, axl.TitForTat()), turns=10, seed=123)               # use axl.Match() for 1v1's against TFT

#player2 = TestStrategy(DEPTH[0])                                              # we may want to run another instance of our search method strategy to train
#battle2 = axl.Match((player2, axl.Cooperator()), turns=10, seed=123)          # against different strategies along with TFT and compare the results
player3 = TestStrategy(DEPTH[0])                                              # the seed=123 helps us replicate the same starting strategy and see how different
battle3 = axl.Match((player3, axl.Defector()), turns=10, seed=123)            # the strategies end up as


result = battle3.play()
print(result)
player3.strategyTable()                                         # prints the table of index, binary, history, and next move of the strategy like in the slides
print(player3.moveProbabilities())

# Tabu Search

In [ ]:
class TestStrategy(Player):
    name = "Test Strategy"
    classifier = {
        "memory_depth": 3,
        "stochastic": False,
        "long_run_time": False,
        "inspects_source": False,
        "manipulates_source": False,
        "manipulates_state": False,
    }

    def __init__(self, memory):
        super().__init__()
        self.memory = memory
        self.strategy_size = 4 ** memory
        self.strategy_list = list(random.choice([Action.C, Action.D]) for i in range(self.strategy_size))           # initial strategy is random
        # self.strategy_list = list(Action.C if i % 2 == 0 else Action.D for i in range(self.strategy_size))        # TFT as a strategy array

    def moveProbabilities(self):
        c_count, d_count = 0.0, 0.0
        for i in self.strategy_list:
            if i == Action.C:
                c_count += 1
            else:
                d_count += 1

        return {'C': c_count / self.strategy_size, 'D': d_count / self.strategy_size}

    def strategyTable(self):                                                    # print the table like in the slides
        offset = self.memory * 2
        i_header = "Index"
        b_header = format("Binary", '^' + str(offset))
        h_header = format("History", '^' + str(offset))
        n_m_header = "Next Move"
        header = f"| {i_header} | {b_header} | {h_header} | {n_m_header} |"
        print(header)
        print("-" * len(header))

        for index,move in enumerate(self.strategy_list):
            binary = format(index, '0' + str(offset) + 'b')                     # convert the index to a string of binary with leading zeros
            history = "".join("C" if i == '0' else "D" for i in binary)         # convert the binary into its history counterpart
            print(f"| {index:>5} | {binary} | {format(history, '>' + str(len(h_header)))} | {move:^9} |")    # print row

    def nextMove(self, opponent):
        fullHist = self.history._plays[-3:] + opponent.history._plays[-3:]      # combine together the last 3 moves of self and opponent
        decimal = 0                                                             # index of the array
        place = 0                                                               # place of the binary value starting from the right
        for i in fullHist[::-1]:                                                # iterates the full history backwards (from right to left)
            if i == D:
                decimal += (2 ** place)                                         # D = 1, C = 0, so we only look at digits with D and add its decimal equivalent
            place += 1                                                          # moves to the next digit (right to left)

        return self.strategy_list[decimal]                                      # returns the next move

    def strategy(self, opponent: Player) -> Action:
      if not self.history:                                                  # very first move
        return C

      if len(self.history) < self.memory:                                   # assigns specific actions for the first few moves
        return D if opponent.history[-1] == C else D                        # the longer the consecutive action is, the more chances of getting stuck in the same move

      return self.nextMove(opponent)

    #def changeStrategy(self):
                # call this method from the strategy() method
                # add your search method here; you can make multiple methods if you'd like
                # no need to return anything since strategy_list is part this class

                # if you want to keep track of what you've changed, you can either:
                # a.) return the index the search method modified and save it outside of this class
                # b.) create a new list at the top of this class like strategy_list and update it here
        #pass    # closes the empty method; remove this when you finish writing the method

    def tabu_search(self, opponent, iterations=100, tabu_list_size=10):
      payoffs = {
          ('C', 'C'): (3, 3),
          ('C', 'D'): (0, 5),
          ('D', 'C'): (5, 0),
          ('D', 'D'): (1, 1),
      }

      # Define the tabu list as an empty list
      tabu_list = []

      # Initialize the current move and the best move
      current_move = random.choice(['C', 'D'])
      best_move = current_move
      best_payoff = payoffs[(current_move, current_move)][0]

      # Start the tabu search algorithm
      for i in range(iterations):
        # Choose the next move randomly from the list of possible moves
        next_move = self.nextMove(opponent)

        # Calculate the expected payoff for the next move
        next_payoff = payoffs[(opponent.history._plays[-1], next_move)][0]

        # Check if the next move is not on the tabu list
        if next_move not in tabu_list:
          # Update the current move and the tabu list
          current_move = next_move
          tabu_list.append(next_move)

          # If the tabu list has reached its maximum size, remove the first item
          if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)

          # Update the best move and the best payoff if necessary
          if next_payoff > best_payoff:
            best_move = next_move
            best_payoff = next_payoff

        self.strategy_list = best_move

      return best_move


player = TestStrategy(DEPTH[0])                                                 # change how far back the strategy will look at (default: DEPTH[0] = 3)
#battle = axl.Match((player, axl.TitForTat()), turns=10, seed=123)               # use axl.Match() for 1v1's against TFT

battle = axl.Match((player, axl.Cooperator()), turns=10, seed=123)          # against different strategies along with TFT and compare the results
#battle = axl.Match((player, axl.Defector()), turns=10, seed=123)            # the strategies end up as


#tournament = axl.Tournament(player, axl.TitForTat(), axl.Cooperator, axl.Defector()) # once the strategy is formed, we compete it with other strategies to find which one gives us the best result in the end
#print(tournament.score)

result = battle.play()
# we may need to put the play() in a for loop depending on the search method
print(result)
#player.strategyTable()                                         # prints the table of index, binary, history, and next move of the strategy like in the slides
print(player.moveProbabilities())
print("final scores (tabu search, strategy): ", battle.final_score())



[(C, C), (D, C), (D, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C), (C, C)]
{'C': 0.578125, 'D': 0.421875}
final scores (tabu search, strategy):  (34, 24)


# Genetic Algorithm




##Individual/Player/Population

In [ ]:
class Individual(Player):
    name = "Individual"
    classifier = {
        "memory_depth": 3,
        "stochastic": False,
        "long_run_time": False,
        "inspects_source": False,
        "manipulates_source": False,
        "manipulates_state": False,
    }

    def __init__(self, memory, id):
        super().__init__()
        self.name = f"Individual {id}"
        self.id = id
        self.memory = memory
        self.strategy_size = 4 ** memory
        self.strategy_list = list(secrets.choice([Action.C, Action.D]) for i in range(self.strategy_size))          # initial strategy is random
        # self.strategy_list = list(Action.C if i % 2 == 0 else Action.D for i in range(self.strategy_size))        # TFT as a strategy array

    def moveCount(self):
        c_count, d_count = 0.0, 0.0
        for i in self.strategy_list:
            if i == Action.C:
                c_count += 1
            else:
                d_count += 1

        return [c_count, d_count, f"{c_count / self.strategy_size} : {d_count / self.strategy_size}"]

    def strategyTable(self):                                                    # print the table like in the slides
        offset = self.memory * 2
        i_header = "Index"
        b_header = format("Binary", '^' + str(offset))
        h_header = format("History", '^' + str(offset))
        n_m_header = "Next Move"
        header = f"| {i_header} | {b_header} | {h_header} | {n_m_header} |"
        print(header)
        print("-" * len(header))

        for index,move in enumerate(self.strategy_list):
            binary = format(index, '0' + str(offset) + 'b')                     # convert the index to a string of binary with leading zeros
            history = "".join("C" if i == '0' else "D" for i in binary)         # convert the binary into its history counterpart
            print(f"| {index:>5} | {binary} | {format(history, '>' + str(len(h_header)))} | {move:^9} |")    # print row

    def resetStrategy(self):
        self.strategy_list = list(secrets.choice([Action.C, Action.D]) for i in range(self.strategy_size))

    def nextMove(self, opponent):
        if len(self.history) < self.memory:
            return secrets.choice([Action.C, Action.D])

        fullHist = self.history[-self.memory:] + opponent.history[-self.memory:]      # combine together the last 3 moves of self and opponent
        decimal = 0                                                             # index of the array
        place = 0                                                               # place of the binary value starting from the right
        for i in fullHist[::-1]:                                                # iterates the full history backwards (from right to left)
            if i == Action.D:
                decimal += (2 ** place)                                         # D = 1, C = 0, so we only look at digits with D and add its decimal equivalent
            place += 1                                                          # moves to the next digit (right to left)

        return self.strategy_list[decimal]                                      # returns the next move

    def strategy(self, opponent: Player) -> Action:
        return self.nextMove(opponent)

##Genetic Algorithm - Search Method

In [ ]:
class GeneticAlgorithm():
    def __init__(self, population_size, memory, sorted=False, reincarnate=False, eliminate=0, versus_TFT=False, versus_Defector=False):
        self.population_size = population_size
        self.chromosomes = 4 ** memory
        self.mutation_prob = 1.0 / self.chromosomes
        self.population = list(Individual(memory, i) for i in range(population_size))

        self.sorted = sorted
        self.leaderboard_initialized = False
        self.reincarnate = reincarnate
        self.eliminate = eliminate

        self.tournament = axl.Tournament(self.population, turns=1, repetitions=1)
        self.versus_TFT = versus_TFT
        self.versus_Defector = versus_Defector

        if versus_TFT:
            self.tft_list = list(axl.Match((p,axl.TitForTat()),turns=1) for p in self.population)
            self.tft_results = [0] * self.population_size

        if versus_Defector:
            self.defector_list = list(axl.Match((p,axl.Defector()),turns=1) for p in self.population)
            self.defector_results = [0] * self.population_size

    def reRank(self, result_tuple):
        index = result_tuple[0]
        for i in self.leaderboard:
            if i[0] > index:
                i[0] -= 1

    def evaluate(self):
        # Play matches
        self.results = self.tournament.play()

        if self.versus_TFT:
            for i,m in enumerate(self.tft_list):
                m.play()
                self.tft_results[i] = m.final_score()[0]

        if self.versus_Defector:
            for i,m in enumerate(self.defector_list):
                m.play()
                self.defector_results[i] = m.final_score()[0]

        # Save leaderboard
        if not self.leaderboard_initialized:
            if not self.sorted:                         # if the population is unsorted, only set up the leaderboard once as the leaderboard will get randomized
                self.leaderboard_initialized = True

            temp_scores = list(s[0] for s in self.results.scores)
            if self.versus_TFT and self.versus_Defector:                # both tft and defector
                initial_leaderboard = list(zip(list(i for i in range(self.population_size)), temp_scores, self.tft_results, self.defector_results))
            elif self.versus_TFT:                                       # only tft
                initial_leaderboard = list(zip(list(i for i in range(self.population_size)), temp_scores, self.tft_results))
            elif self.versus_Defector:                                  # only defector
                initial_leaderboard = list(zip(list(i for i in range(self.population_size)), temp_scores, self.defector_results))
            else:                                                       # neither
                initial_leaderboard = list(zip(list(i for i in range(self.population_size)), temp_scores))

            self.leaderboard = list(list(initial_leaderboard[i]) for i in range(self.population_size))   # convert tuples into lists

        # Sort or unsort data
        if self.population_size == 1 or (self.reincarnate and self.eliminate <= 0):
            return False               # leave at least 1 individual, or reincarnate but no count given

        if self.versus_TFT and self.versus_Defector:
            self.leaderboard.sort(reverse=True, key=lambda x:x[3])
            self.leaderboard.sort(reverse=True, key=lambda x:x[2])
            self.leaderboard.sort(reverse=True, key=lambda x:x[1])
        elif self.versus_TFT or self.versus_Defector:
            self.leaderboard.sort(reverse=True, key=lambda x:x[2])
            self.leaderboard.sort(reverse=True, key=lambda x:x[1])
        else:
            self.leaderboard.sort(reverse=True, key=lambda x:x[1])


        if self.reincarnate:     # reincarnate worst individuals
            for i in range(self.eliminate):
                self.population[self.leaderboard[-1][0]].resetStrategy()
        elif self.eliminate > 0:                        # no reincarnation, but eliminate worst individuals
            if self.population_size - self.eliminate < 1:
                for i in range(self.eliminate-1, 0, -1):
                    if self.population_size - i == 1:
                        self.eliminate = i
                        break

            self.population_size -= self.eliminate

            for i in range(self.eliminate):
                self.population.remove(self.population[self.leaderboard[-1][0]])
                remove_ranking = self.leaderboard[-1]
                self.leaderboard.remove(remove_ranking)
                self.reRank(remove_ranking)

                if self.versus_TFT:
                    self.tft_list.remove(self.tft_list[remove_ranking[0]])
                    self.tft_results.remove(self.tft_results[remove_ranking[0]])

                if self.versus_Defector:
                    self.defector_list.remove(self.defector_list[remove_ranking[0]])
                    self.defector_results.remove(self.defector_results[remove_ranking[0]])

        # Crossover & Mutate
        size = self.population_size
        if size % 2 != 0:                                   # if population size is odd, the last remaining will not cross over with anyone
            size -= 1

        if not self.sorted:                                 # randomize the index list for random pairing
            random.shuffle(self.results.ranking)

            for i in range(0, size, 2):                     # pair every 2 individuals in sequence (using the randomized list)
                individual_A, individual_B = self.population[self.results.ranking[i]], self.population[self.results.ranking[i+1]]
                self.crossOver(individual_A.strategy_list, individual_B.strategy_list)
        else:
            for i in range(0, size, 2):                     # pair every 2 individuals in sequence (using the sorted list)
                individual_A, individual_B = self.population[self.leaderboard[i][0]], self.population[self.leaderboard[i+1][0]]
                self.crossOver(individual_A.strategy_list, individual_B.strategy_list)

        return True

    def mutate(self):                                   # a chance of 1/strategy size of mutating
        return random.choices((True, False), (self.mutation_prob, 1-self.mutation_prob))[0]

    def crossOver(self, strategy_A, strategy_B):
        for i in range(self.chromosomes):
            if secrets.choice((True, False)):                                   # 50% chance to swap/crossover an element
                strategy_A[i], strategy_B[i] = strategy_B[i], strategy_A[i]

        if self.mutate():                                                       # see if strategy_A wants to mutate
            random_index = random.randint(0, self.chromosomes-1)
            strategy_A[random_index].flip()

        if self.mutate():                                                       # see if strategy_B wants to mutate
            random_index = random.randint(0, self.chromosomes-1)
            strategy_B[random_index].flip()

    def getFirst(self):
        return self.population[self.leaderboard[0][0]]

    def getSecond(self):
        return self.population[self.leaderboard[1][0]]

    def getThird(self):
        return self.population[self.leaderboard[2][0]]

##Running the code

In [ ]:
C, D = Action.C, Action.D

ITERATIONS = [1000, 10000]  # Use ITERATIONS[0] for now; ITERATIONS is the number of times we cycle between search method and playing against TFT
TURNS = 100                 # Number of turns for the match between search method strategy vs TFT
DEPTH = [3, 4, 5]           # Use DEPTH[0] for now

GENERATION = ITERATIONS[0]
POPULATION_SIZE = [100, 1000, 10000]

individual_count = 10
ga_list = list(GeneticAlgorithm(population_size=POPULATION_SIZE[0], memory=DEPTH[0], sorted=True, reincarnate=False, eliminate=1, versus_TFT=False, versus_Defector=False) for i in range(individual_count))

start = time.time()
for ga in ga_list:
    for gen in range(GENERATION):
        if not ga.evaluate():
            break
end = time.time()

individuals_list = list(ga.population[0] for ga in ga_list)
for i in individuals_list:
    print(i.strategy_list)

vs_TFT_list = list(axl.Match((i, axl.TitForTat()), turns=100) for i in individuals_list)

for match in vs_TFT_list:
    match.play()

vs_Def_list = list(axl.Match((i, axl.Defector()), turns=100) for i in individuals_list)

for match in vs_Def_list:
    match.play()

move_counts = list(individual.moveCount() for individual in individuals_list)

data_TFT = list(list(vs_TFT_list[i].final_score()) + move_counts[i] for i in range(individual_count))
data_Def = list(list(vs_Def_list[i].final_score()) + move_counts[i] for i in range(individual_count))

df_1 = pd.DataFrame(data_TFT, columns=["Player Score", "TFT Score", "Cooperates", "Defects", "C/D Ratio"])
df_2 = pd.DataFrame(data_Def, columns=["Player Score", "Defector Score", "Cooperates", "Defects", "C/D Ratio"])

path = "Results.xlsx"
with pd.ExcelWriter(path) as writer:
    df_1.to_excel(writer, sheet_name=f"TFT {SHEETS}")
    df_2.to_excel(writer, sheet_name=f"Defector {SHEETS}")

SHEETS += 1

print(f"Start: {time.ctime(start)}")
print(f"End: {time.ctime(end)}")

# future work:  Take first three best strategies from n different instances of GeneticAlgorithm() and run their respective evaluation and crossover
#               Mix and combine GA with other search methods